In [3]:
import numpy as np
from sklearn.cluster import KMeans
from sklearnex import patch_sklearn; patch_sklearn()
import scipy
import pandas as pd
from concurrent.futures import ProcessPoolExecutor as Executor
from plotly import graph_objects as go, subplots as sp

axis_col = 'rgba(0, 0, 0, 0.15)'
zero_col = 'rgba(0, 0, 0, 0.3)'
no_col = 'rgba(0, 0, 0, 0)'

xaxis_desc: dict = dict(linecolor=no_col, gridcolor=axis_col, zerolinecolor=zero_col, zerolinewidth=2)
yaxis_desc: dict = dict(linecolor=no_col, gridcolor=axis_col, zerolinecolor=zero_col, zerolinewidth=2)
layout = dict(
    autosize=True,
    width=1400,
    height=400,
    margin=dict(
        l=60, r=25, b=60, t=60, pad=5
    ),
    # paper_bgcolor="white",
    font_family="Times New Roman",
    font_color="black",
    font_size=20,
    plot_bgcolor='white',
    xaxis=dict(**xaxis_desc, ),
    yaxis=yaxis_desc,
)

from local.caching import load, save

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
ALPHAS = [2, 2, 2]
NU = 2
BETA = 2
NUM_CLUSTERS = len(ALPHAS)

_RAND_STATE = 0

np.random.seed(_RAND_STATE)
xs = pd.read_csv('pmm_q1.tsv', header=None).to_numpy()
alphas = np.array(ALPHAS)
alpha_sum = alphas.sum()

# kmeans initialization
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=_RAND_STATE)
kmeans.fit(xs)
assignments = np.zeros(shape=(xs.shape[0], NUM_CLUSTERS))
initial_lambdas = np.zeros(shape=(NUM_CLUSTERS, ))
pi = np.zeros(shape=(NUM_CLUSTERS, ))
for k in range(NUM_CLUSTERS):
    group = np.array([x for a, x in zip(kmeans.labels_, xs) if a == k])
    initial_lambdas[k] = group.mean()
    pi[k] = group.shape[0]/xs.shape[0]

/home/tony/Utilities/mambaforge/envs/445/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [32]:
previous = False
retry=False
retry = True
iterations = 200

save_file = f"em_{iterations}"
if not retry:
    try:
        pi, current_lambdas, log_lls = load(save_file)
        previous = True
    except FileNotFoundError:
        pass

if not previous or retry:
    current_lambdas = initial_lambdas.copy()
    ll_last_component = np.zeros(shape=assignments.shape)
    log_lls = []
    for it in range(iterations):
        print(f"iteration: {it+1}", end='\r')
        xvec = xs[:, 0]

        # e-step
        gammas = []
        for k in range(NUM_CLUSTERS):
            gammas.append(scipy.stats.gamma.pdf(current_lambdas[k], NU))
            for i, x in enumerate(xvec):
                fx_theta = scipy.stats.poisson.pmf(x, current_lambdas[k])
                assignments[i, k] = pi[k]*fx_theta
                ll_last_component[i, k] = np.log(pi[k]) + np.log(fx_theta)

        assignments = (assignments.T/assignments.sum(axis=1)).T
        log_ll = np.log(scipy.stats.dirichlet.pdf(pi, alphas)) + np.log(gammas).sum() + (assignments*ll_last_component).sum()
        # print(scipy.stats.dirichlet.pdf(pi, alphas))
        log_lls.append(log_ll)

        # m-step
        for k in range(NUM_CLUSTERS):
            K, N = NUM_CLUSTERS, xs.shape[0]
            assignments_k_sum = assignments[:, k].sum()

            pi[k] = (alphas[k] - 1 + assignments_k_sum) / (alpha_sum - K + N)
            current_lambdas[k] = ((assignments[:, k]*xvec).sum()+1) / (assignments_k_sum+2)
    
    save(save_file, (pi, current_lambdas, log_lls))

print(f"π: {pi}")
print(f"λ: {current_lambdas.T}")

compressing & caching data to [{WORKSPACE}/hw/04/cache/em_200.pkl.gz]
π: [0.48632503 0.25056332 0.26311166]
λ: [ 0.73564205 12.48231551  5.6132459 ]


In [33]:
max_x = xs.max()
min_x = 0

fig = go.Figure()
cutt = None
fig.add_trace(
    go.Scatter(
        x = [x for x, y in enumerate(log_lls[:cutt])],
        y = [y for x, y in enumerate(log_lls[:cutt])],
        mode="lines+markers",
        marker=dict(size=3),
        line=dict(width=1),
        showlegend=False,
    ),
)

fig.update_annotations(font_size=24)
fig.update_layout(go.Layout(layout))

fig.show()
# from IPython.display import Image
# Image(filename='q5_FCGR3A.png')

In [7]:
max_x = xs.max()
min_x = 0

fig = go.Figure()
for lmd in current_lambdas.T:
    distr = [(x, scipy.stats.poisson.pmf(x, lmd)) for x in np.arange(min_x, max_x+1, 1)]
    fig.add_trace(
        go.Scatter(
            x = [x for x, y in distr],
            y = [y for x, y in distr],
            mode="lines+markers",
            showlegend=False,
        ),
    )

fig.update_annotations(font_size=24)
fig.update_layout(go.Layout(layout))

fig.show()
# from IPython.display import Image
# Image(filename='q5_FCGR3A.png')

In [20]:
current_lambdas = initial_lambdas.copy()
iterations = 100

def sample_gamma(alpha, beta):
    return scipy.stats.gamma(alpha).rvs(size=1)[0]

# kmeans initialization
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=_RAND_STATE, n_init="auto")
kmeans.fit(xs)
assignments = np.zeros(shape=(xs.shape[0], NUM_CLUSTERS))
initial_lambdas = np.zeros(shape=(NUM_CLUSTERS, ))
pi = np.zeros(shape=(NUM_CLUSTERS, ))
for k in range(NUM_CLUSTERS):
    group = np.array([x for a, x in zip(kmeans.labels_, xs) if a == k])
    initial_lambdas[k] = group.mean()
    pi[k] = group.shape[0]/xs.shape[0]

for it in range(iterations):
    print(f"iteration: {it+1}", end='\r')
    
    Z = []
    for x in xs:
        _Z = []
        for k in range(NUM_CLUSTERS):
            za = pi[k] * scipy.stats.poisson.pmf(x, current_lambdas[k])[0]
            zb = (pi*np.array([scipy.stats.poisson.pmf(x, current_lambdas[k]) for k in range(NUM_CLUSTERS)]).T[0]).sum()
            z_k = za/zb
            _Z.append(z_k)
            assert z_k <1, (za, zb)

        theta = scipy.stats.multinomial(1, _Z) # 1 trial, prob. success is Z
        z = [i for i, z in enumerate(theta.rvs(size=1)[0]) if z > 0]
        assert len(z) == 1
        z = z[0]
        Z.append(z)

    lambdas = []
    paramaters = []
    for k in range(NUM_CLUSTERS):
        z_k = [x for x, z in zip(xs, Z) if z == k]
        # print(NU + np.sum(z_k), BETA + len(z_k))
        lambdas.append(sample_gamma(NU + np.sum(z_k), BETA + len(z_k)))
        paramaters.append(alphas[k] + len(z_k))

    pi = scipy.stats.dirichlet.rvs(paramaters, size=1)[0]

In [22]:
lambdas

[369.56532230593484, 2313.8943833862977, 2255.513940142632, 2245.6789804696036]

In [10]:
asdf.rvs(10)

NameError: name 'asdf' is not defined

In [ ]:
ALPHAS = [2, 2, 2]
NU = 2
BETA = 2
NUM_CLUSTERS = len(ALPHAS)

_RAND_STATE = 0

np.random.seed(_RAND_STATE)
xs = pd.read_csv('pmm_q1.tsv', header=None).to_numpy()
alphas = np.array(ALPHAS)
alpha_sum = alphas.sum()

assignments = np.zeros(shape=(xs.shape[0], NUM_CLUSTERS))
initial_lambdas = np.array([0.6, 1])

current_lambdas = initial_lambdas.copy()
log_lls = []
for it in range(iterations):
    print(f"iteration: {it+1}", end='\r')
    xvec = xs[:, 0]

    # e-step
    gammas = []
    for k in range(NUM_CLUSTERS):
        gammas.append(scipy.stats.gamma.pdf(current_lambdas[k], NU))
        for i, x in enumerate(xvec):
            assignments[i, k] = pi[k]*scipy.stats.poisson.pmf(x, current_lambdas[k])

    assignments = (assignments.T/assignments.sum(axis=1)).T
    log_ll = np.log(scipy.stats.dirichlet.pdf(pi, alphas)) + np.log(gammas).sum() + np.log(assignments.max(axis=1)).sum()
    log_lls.append(log_ll)

    # m-step
    for k in range(NUM_CLUSTERS):
        K, N = NUM_CLUSTERS, xs.shape[0]
        assignments_k_sum = assignments[:, k].sum()

        pi[k] = (alphas[k] - 1 + assignments_k_sum) / (alpha_sum - K + N)
        current_lambdas[k] = ((assignments[:, k]*xvec).sum()+1) / (assignments_k_sum+2)